# 2.5 Automatic Differentiation
## 2.5.1 A Simple Function

In [6]:
import torch

x = torch.arange(4.0)
print("x: ", x)

x.requires_grad_(True)
print("x.grad: ", x.grad)

y = 2 * torch.dot(x, x) # y = 2 * xT * x
print("y: ", y)
y.backward()
print("x.grad: ", x.grad) # x.grad = dy/dx = 4x

x.grad.zero_() # reset x.grad to zero
y = x.sum()
y.backward()
print("x.grad: ", x.grad) # x.grad = dy/dx = 1

x:  tensor([0., 1., 2., 3.])
x.grad:  None
y:  tensor(28., grad_fn=<MulBackward0>)
x.grad:  tensor([ 0.,  4.,  8., 12.])
x.grad:  tensor([1., 1., 1., 1.])


## 2.5.2 Backward for Non-Scalar Variables

In [13]:
x.grad.zero_()
y = x * x
print("y: ", y)
y.backward(gradient=torch.ones(len(y))) # faster way: y.sum().backward()
print("x.grad: ", x.grad)

y:  tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)
x.grad:  tensor([0., 2., 4., 6.])


## 2.5.3 Detaching Computation

In [15]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
print("x.grad: ", x.grad) # x.grad = dz/dx = d/dz (u * x) = u (do not flow back to x)

x.grad:  tensor([0., 1., 4., 9.])


## 2.5.4 Gradients and Python Control Flow

In [24]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
print("a: ", a)
print("d: ", d)
d.backward()
print("a.grad: ", a.grad) # a.grad = dd/da

a:  tensor(-0.1513, requires_grad=True)
d:  tensor(-123972.5078, grad_fn=<MulBackward0>)
a.grad:  tensor(819200.)
